# alchemy_sql_query

Execute arbitrary SQL queries against a remote RDBMS supported by python's SQLAlchemy library

In [1]:
!pip install sqlalchemy==1.4.29 pandas==1.3.5

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import glob
import logging
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import pandas as pd
import os
import shutil
import sys
import re

In [3]:
# type of database server (sqlalchemy dialect), e.g. postgresql
db_type = os.environ.get('db_type', 'postgresql')

# hostname of database server
host = os.environ.get('host')

# database name
database = os.environ.get('database')

# db user
user = os.environ.get('user')

# db password
password = os.environ.get('password')

# db port
port = int(os.environ.get('port', 5432))

# SQL statement
sql = os.environ.get('sql')

In [4]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))


for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [5]:
logging.warning(sql)

In [6]:
engine = create_engine(f'{db_type}://{user}:{password}@{host}:{port}/{database}')
with engine.connect() as con:
    con.execute(text(sql))